In [ ]:
def calculate_size(dim, kernel_size, padding, stride):
    return ((dim - kernel_size + 2 * padding) / stride) + 1

In [ ]:
s1 = calculate_size(256, 3, 0, 1)  # Conv2d
s2 = calculate_size(s1, 2, 0, 2)  # AvgPool2d
s3 = calculate_size(s2, 3, 0, 1)  # Conv2d
s4 = calculate_size(s3, 3, 2, 2)  # MaxPool2d
s5 = calculate_size(s4, 3, 0, 1)  # Conv2d
s6 = calculate_size(s5, 2, 0, 2)  # MaxPool2d
s6**2 * 16

In [ ]:
from torch import nn

In [ ]:
layer = nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, padding=0, stride=1)

In [ ]:
layer.kernel_size[0]

In [ ]:
from dataclasses import dataclass

In [ ]:
@dataclass
class Human:
    name: str
    age: int
    has_dog: bool

    def say_hi(self):
        print("Hello, my name is", self.name)

In [ ]:
Human("Nick", 22, True).say_hi()

In [ ]:
import torch

In [ ]:
torch.cat([torch.zeros(5), torch.ones(5)])

In [ ]:
x = {"a": 0, "b": 1}

In [ ]:
dict(zip(x.keys(), [2, 3]))

In [ ]:
[[]] * 3

In [ ]:
torch.Tensor().mean()